# Project Part 2: Amazon Beauty Products

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# path='/content/drive/Shareddrives/Introduction to Data Science/data/'
path=""

## 1.0 Introduction

Importing some libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import datetime
import matplotlib
import json
from scipy.stats import gaussian_kde
from scipy.stats import zscore
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler 

## 2.0 Loading Datasets 

Getting familiar with the data we have

### 2.1 meta_All_Beauty.json

This dataset contains all the product details

In [ ]:
import json
import numpy as np
import pandas as pd

product = []
with open( 'meta_All_Beauty.json') as f:
    for line in f:
        product.append(json.loads(line))

In [ ]:
raw_product_table = pd.DataFrame(product)
raw_product_table.head(10)

### 2.2 Review_Beauty.json

This file contains all the review given by customers.

In [ ]:
import json
import numpy as np
import pandas as pd

review = []
with open('Review_Beauty.json') as f:
    for line in f:
        review.append(json.loads(line))

In [ ]:
raw_review_table = pd.DataFrame(review)
raw_review_table.head(10)

### 2.3 All_Beauty.csv

csv file provide by lecturer.

In [ ]:
header_names = ['ProductID', 'UserID', 'Review', 'Date']
review_simplified = pd.read_csv(path + 'All_Beauty.csv', header=None, names=header_names)
review_simplified.head(50)

In [ ]:
review_simplified.shape

### 2.4 Luxury_Beauty.json

Additional dataset that is used mainly for predictive modelling part. This file contains all the review given by customers for luxury beauty product

In [ ]:
import json
import numpy as np
import pandas as pd

luxuryBeautyReview = []
with open('Luxury_Beauty.json') as f:
    for line in f:
        luxuryBeautyReview.append(json.loads(line))

In [ ]:
raw_luxuryBeautyReview = pd.DataFrame(luxuryBeautyReview)
raw_luxuryBeautyReview

### 2.5 meta_Luxury_Beauty.json

Additional dataset that is used mainly for predictive modelling part. This file contains all the luxury beauty product details.

In [ ]:
import json
import numpy as np
import pandas as pd

luxuryBeautyProduct = []
with open('meta_Luxury_Beauty.json') as f:
    for line in f:
        luxuryBeautyProduct.append(json.loads(line))

In [ ]:
raw_luxuryBeautyProduct = pd.DataFrame(luxuryBeautyProduct)
raw_luxuryBeautyProduct

## 3.0 Data Cleaning

Cleaning our data, which includes the steps below:

Step 1 : Remove duplicates or irrelevant observations

Step 2 : Fix structural errors

Step 3 : Identify and handle outliers

Step 4 : Handle missing data


### 3.1 Cleaning meta_All_Beauty.json

Which is now loaded into Product (a DataFrame)

In [ ]:
#drop irrelevant observations
product_table = raw_product_table.drop(['category','tech1','description', 'fit','image','tech2','feature','main_cat','similar_item','date'], axis=1)


In [ ]:
# some ASIN inside details has space after 
# example  'ASIN: ' vs 'ASIN:'
# but going to drop 'details' because didn't use it
# this is to check whether the asin and the ASIN inside details is correct

count = 0
index_to_drop = [] 

for i in range(len(product_table.index)):
    dct = product_table['details'].iloc[i]
    if 'ASIN: ' in dct: 
        asin = dct['ASIN: ']
    elif 'ASIN:' in dct:
        asin = dct['ASIN:']
    if asin != product_table['asin'].iloc[i]:
        index_to_drop.append(i)
        count += 1
        
print(count)

In [ ]:
##drop irrelevant observations
product_table = raw_product_table.drop(['category','tech1','description','fit','image','tech2','brand','feature','main_cat','similar_item','date','details'],axis = 1)

##drop duplicates
#no duplicate asins and row
product_table.drop_duplicates(['asin'],inplace = True)
product_table.reset_index(drop=True, inplace=True)

##fix structural errors

#for column 'rank' 
#remove uneccessary symbol like comma ',' and words
#convert the string into integer 
product_table['rank'].replace(',','', regex=True, inplace=True)
product_table['rank'] = product_table['rank'].str.extract('(\d+)')

#for column 'price'
#if 'price' don't start with $ or has '-' means null value, fillna with 'unknown'
#remove uneccessary symbol like comma ',' and '$'
product_table.loc[product_table["price"].str[0] != "$", "price"] = "unknown"
product_table.loc[product_table["price"].str.contains('-'), "price"] = "unknown"
product_table['price'].replace("\$","", regex=True, inplace=True)
product_table['price'].replace(',','', regex=True, inplace=True)

##deal with missing data
#missing data in 'rank' and 'price'
#product_table.replace('\[]','', regex=True, inplace=True)
product_table['rank'].fillna(0,inplace = True)
product_table['price'].replace("unknown",0, regex=True, inplace=True)
product_table.fillna("unknown",inplace = True)

##changing data type
product_table['rank']= product_table['rank'].astype(int)
product_table['price'] = product_table['price'].apply(pd.to_numeric)

##Identify and handle outliers
z_scores = stats.zscore(pd.DataFrame(product_table['rank']))
abs_z_scores = np.abs(z_scores)
outlier = (abs_z_scores > 3).all(axis=1)
product_table.drop(product_table[outlier].index, inplace = True)

product_table.shape

In [ ]:
unique = product_table["asin"].value_counts()
unique

### 3.2 Cleaning Review_Beauty.json
Which is now loaded into review (a DataFrame)

In [ ]:
##drop irrelevant observations
review_table = raw_review_table.drop(['reviewerName','reviewText','summary','style','image'],axis = 1)

##drop duplicates
review_table.drop_duplicates(inplace = True)
review_table.reset_index(drop=True, inplace=True)

##fix structural errors
#for column 'vote'
#remove uneccessary symbol ','
review_table['vote'].replace(',','', regex=True, inplace=True)
test_review_table = review_table.copy(deep=True)

##deal with missing data
review_table['vote'].fillna(0,inplace = True)
review_table.fillna('unknown',inplace = True)

##changing data type
review_table['vote'] = review_table['vote'].astype(int)

#separate reviewTime into three different columns
review_table['reviewTime'].replace(',','', regex=True, inplace=True)
date = review_table['reviewTime'].str.split(' ', n=2,expand = True)
date.columns =['month','day','year']

#drop reviewTime column
review_table.drop(['reviewTime'],axis = 1,inplace = True)

#concat both of the tables
review_table = pd.concat([review_table,date], axis=1)

#rearrange sequence of the columns
cols = review_table.columns.tolist()
cols = cols[-1:] + cols[-3:-1] + cols[:-3]
review_table = review_table[cols]

review_table

### 3.3 Cleaning Luxury_Beauty.json
Cleaning luxury beauty product review datasets

In [ ]:
##drop irrelevant observations
luxuryBeautyReview_table = raw_luxuryBeautyReview.drop(['reviewerName','reviewText','summary','style','image'],axis = 1)

##drop duplicates
luxuryBeautyReview_table.drop_duplicates(inplace = True)
luxuryBeautyReview_table.reset_index(drop=True, inplace=True)

##fix structural errors
#for column 'vote'
#remove uneccessary symbol ','
luxuryBeautyReview_table['vote'].replace(',','', regex=True, inplace=True)
#test_review_table = review_table.copy(deep=True)

##deal with missing data
luxuryBeautyReview_table['vote'].fillna(0,inplace = True)
luxuryBeautyReview_table.fillna('unknown',inplace = True)

##changing data type
luxuryBeautyReview_table['vote'] = luxuryBeautyReview_table['vote'].astype(int)

#separate reviewTime into three different columns
luxuryBeautyReview_table['reviewTime'].replace(',','', regex=True, inplace=True)
date_luxuryBeautyReview = luxuryBeautyReview_table['reviewTime'].str.split(' ', n=2,expand = True)
date_luxuryBeautyReview .columns =['month','day','year']

#drop reviewTime column
luxuryBeautyReview_table.drop(['reviewTime'],axis = 1,inplace = True)

#concat both of the tables
luxuryBeautyReview_table = pd.concat([luxuryBeautyReview_table,date_luxuryBeautyReview ], axis=1)

#rearrange sequence of the columns
cols_LB = luxuryBeautyReview_table.columns.tolist()
cols_LB = cols_LB[-1:] + cols_LB[-3:-1] + cols_LB[:-3]
luxuryBeautyReview_table = luxuryBeautyReview_table[cols_LB]

luxuryBeautyReview_table

### 3.4 Cleaning meta_Luxury_Beauty.json
Cleaning luxury beauty product datasets

In [ ]:
##drop irrelevant observations
luxuryBeautyProduct_table = raw_luxuryBeautyProduct.drop(['category','tech1','description','fit','image','tech2','brand','feature','main_cat','similar_item','date','details'],axis = 1)

##drop duplicates
#no duplicate asins and row
luxuryBeautyProduct_table.drop_duplicates(['asin'],inplace = True)
luxuryBeautyProduct_table.reset_index(drop=True, inplace=True)

##fix structural errors

#for column 'rank' 
#remove uneccessary symbol like comma ',' and words
#convert the string into integer 
luxuryBeautyProduct_table['rank'].replace(',','', regex=True, inplace=True)
luxuryBeautyProduct_table['rank'] = luxuryBeautyProduct_table['rank'].str.extract('(\d+)')

#for column 'price'
#if 'price' don't start with $ or has '-' means null value, fillna with 'unknown'
#remove uneccessary symbol like comma ',' and '$'
luxuryBeautyProduct_table.loc[luxuryBeautyProduct_table["price"].str[0] != "$", "price"] = "unknown"
luxuryBeautyProduct_table.loc[luxuryBeautyProduct_table["price"].str.contains('-'), "price"] = "unknown"
luxuryBeautyProduct_table['price'].replace("\$","", regex=True, inplace=True)
luxuryBeautyProduct_table['price'].replace(',','', regex=True, inplace=True)

##deal with missing data
#missing data in 'rank' and 'price'
#product_table.replace('\[]','', regex=True, inplace=True)
luxuryBeautyProduct_table['rank'].fillna(0,inplace = True)
luxuryBeautyProduct_table['price'].replace("unknown",0, regex=True, inplace=True)
luxuryBeautyProduct_table.fillna("unknown",inplace = True)

##changing data type
luxuryBeautyProduct_table['rank']= luxuryBeautyProduct_table['rank'].astype(int)
luxuryBeautyProduct_table['price'] = luxuryBeautyProduct_table['price'].apply(pd.to_numeric)

##Identify and handle outliers
z_scores_LB = stats.zscore(pd.DataFrame(luxuryBeautyProduct_table['rank']))
abs_z_scores_LB = np.abs(z_scores_LB)
outlier_LB = (abs_z_scores_LB > 3).all(axis=1)
luxuryBeautyProduct_table.drop(luxuryBeautyProduct_table[outlier_LB].index, inplace = True)

luxuryBeautyProduct_table

## 4.0 Data Transformation

### 4.1 Normalization

In [ ]:
#min-max normalization
product_table['rank'] = (product_table['rank'] - product_table['rank'].min())/(product_table['rank'].max()-product_table['rank'].min())
product_table

### 4.2 Merging
Merge review_table with product_table

#### 4.2.1 Merging review_table and product_table
Merging the two tables.

In [ ]:
# Outer merge of review_table & product_table
dfMergedOuter = (pd.merge(product_table,review_table, on='asin', how='outer', indicator=True)).set_index('asin')

# Outer left (product_table), to find out the product that doesn't have any review
dfMergedOuterProduct=dfMergedOuter[dfMergedOuter['_merge']=='left_only']
dfMergedOuterProduct

In [ ]:
# Outer merge of review_table & product_table
dfMergedOuter2 = pd.merge(review_table, product_table, on='asin', how='outer', indicator=True)

# Outer left (review_table), to find out the product review that doesn't exist in product_table
dfMergedOuterReview=dfMergedOuter2[dfMergedOuter2['_merge']=='left_only']
dfMergedOuterReview

In [ ]:
# to find the intersection betweeen both tables
dfMergedInner = (pd.merge(product_table,review_table, on='asin', how='inner', indicator=True)).set_index('asin')
dfMergedInner.reset_index(inplace = True)
dfMergedInner

After merging both tables, we found that there are 2 existing products without any reviews. Hence we drop them from product_table.

For review_table, there are 5120 reviews on products that were not in our dataset, we drop them too.

In [ ]:
# Drop both products since they do not have any reviews
product_table.drop(product_table[product_table['asin']=='B000NICBSI'].index, inplace = True) 
product_table.drop(product_table[product_table['asin']=='B01HHYC4UA'].index, inplace = True) 

In [ ]:
# Update our review table using the inner merge
final_review_table = dfMergedInner.drop(['title','also_buy','rank','also_view','price','reviewerID', '_merge'],axis = 1)
final_review_table

After cleaning and merging the data, 32485 beauty products and 357283 reviews are remained in our dataset.


#### 4.2.2 Final Merged Table
Final merged table has been prepared for future use in data processing

In [ ]:
dfMergedInner



We will be using `product_table` and `final_review_table` for our data analysis.

`product_table`: Contains 32485 unique beauty products selling on Amazon. All listed product have at least one review in our dataset. It contains sales rank and/or price that is 0.

`final_review_table`: Contains 357283 product reviews. All recorded reviews have a matching product in our dataset.

### 4.3 Useful Functions

#### 4.3.1 process_merged_review

In [ ]:
# calculates average rating of each product
# -----------------------------------------
# 1 parameter:
# review_arg['asin', 'overall']
# -----------------------------------------
# return:
# avg_review['asin', 'average-rating']
# -----------------------------------------

def avgReview(review_arg):
  avg_review = review_arg.groupby(['asin']).mean()
  avg_review.rename(columns={'overall':'average-rating'},inplace=True)
  avg_review.reset_index(inplace = True)
  return avg_review

In [ ]:
# calculates number of reviews of each product
# --------------------------------------------
# 1 parameter:
# review['asin', 'overall']
# --------------------------------------------
# return:
# count_review ['asin', 'number-of-review']
# --------------------------------------------

def countReview(review_arg):
  count_review = review_arg.groupby(['asin']).count()
  count_review.rename(columns={'overall':'number-of-review'},inplace=True)
  count_review.reset_index(inplace = True)
  return count_review

In [ ]:
# merge 3 dataframe together
# --------------------------------------------
# 3 paramters:
# avg_review_arg ['asin', 'average-rating']
# count_review_arg ['asin', 'number-of-review']
# product_rank_arg ['asin','rank']
# --------------------------------------------
# result:
# merged_review_table ['asin', 'average-rating', 'number-of-review', 'rank']
# --------------------------------------------

def merge_avgReview_numReview_product(avg_review_arg, count_review_arg, product_rank_arg):
# merge average rating score
  merged_review_table = (pd.merge(product_rank_arg, avg_review_arg, on='asin', how='inner', indicator=True)).set_index('asin')
  merged_review_table.reset_index(inplace=True)
  merged_review_table = merged_review_table.drop(['_merge'],axis = 1)
  merged_review_table.rename(columns={'overall':'average-rating'},inplace=True)

  # merge number of reviews
  merged_review_table = (pd.merge(merged_review_table, count_review_arg, on='asin', how='inner', indicator=True)).set_index('asin')
  merged_review_table.reset_index(inplace = True)
  merged_review_table = merged_review_table.drop(['_merge'],axis = 1)
  merged_review_table.rename(columns={'overall':'number-of-review'},inplace=True)

  # filter out product with 0 as sales rank (32083 products left)
  merged_review_table = merged_review_table[merged_review_table['rank'] > 0]
  merged_review_table.sort_values(by=['rank']) 

  return merged_review_table

In [ ]:
# calculate avg_review and count_review, and then merge with product_rank
# --------------------------------------------
# 2 parameters:
# review_arg ['asin', 'overall']
# product_rank_arg ['asin','rank']
# --------------------------------------------
# return:
# merged_review_table ['asin', 'rank', 'average-rating, 'number-of-review']
# --------------------------------------------

def process_merged_review(review_arg, product_rank_arg):
    avg_review = avgReview(review_arg)
    count_review = countReview(review_arg)
    merged_review_table = merge_avgReview_numReview_product(avg_review, count_review, product_rank_arg)
    return merged_review_table

#### 4.3.2 process_verified_review

In [ ]:
# calculates number of verified reviews of each product
# --------------------------------------------
# 2 parameters:
# verified_review ['asin', 'overall', 'verified']
# merged_review_table
# --------------------------------------------
# return:
# mv_review_table ['asin', 'rank', 'average-rating, 'number-of-review',
# 'average-verified-rating', 'verified']
# --------------------------------------------

def process_verified_review(verified_review, merged_review_table_arg):
  # gets verified reviews' average score (left 30430 rows)
  avg_verified_review = verified_review[verified_review['verified'] == True]
  avg_verified_review = avg_verified_review.groupby(['asin']).mean()
  avg_verified_review.rename(columns={'overall':'average-verified-rating'},inplace=True)
  avg_verified_review.drop(['verified'],axis=1,inplace=True)
  avg_verified_review.reset_index(inplace = True)
  # merge merged_review_table && avg_verified_review
  copy_merged_review = merged_review_table_arg.copy()
  #mv_review_table = (pd.merge(merged_review_table_arg.copy(), avg_verified_review, on='asin', how='left', indicator=True).fillna(0)).set_index('asin')
  copy_merged_review['average-verified-rating'] = copy_merged_review.asin.map(avg_verified_review\
                                                                             .set_index('asin')['average-verified-rating'])\
                                                                             .fillna(0).astype(float)
  # return
  return copy_merged_review

Generate merged_review_table

In [ ]:
review = final_review_table[['asin', 'overall']]
product_rank = product_table[['asin','rank']]
merged_review_table = process_merged_review(review, product_rank)
merged_review_table

Generate mv_review_table

In [ ]:
vr = final_review_table[['asin', 'overall', 'verified']]
mv_review_table = process_verified_review(vr, merged_review_table)
mv_review_table

Here we have two processed tables:

`merged_review_data`: All cleaned products with average rating, total number of reviews and more.

`mv_review_table`: The same product table, but only verified reviews are considered.


In [ ]:
merged_review_table.corr()

#### 4.3.3 time frame

In [ ]:
def filterByYear(x, data):
    result = data['year'] == x
    result = data[result]
    return result

In [ ]:
# max date review groupby 'asin'
product_max_date = final_review_table.groupby(['asin'], sort=True)['year'].max()
product_max_date = pd.DataFrame(product_max_date)
product_max_date.reset_index(level=0, inplace=True)

# min date review groupby 'asin'
product_min_date = final_review_table.groupby(['asin'], sort=True)['year'].min()
product_min_date = pd.DataFrame(product_min_date)
product_min_date.reset_index(level=0, inplace=True)

In [ ]:
# merge max and min together
# rename column and reset index
product_min_max_date = (pd.merge(product_max_date, product_min_date, on='asin', how='inner', indicator=True)).set_index('asin')
product_min_max_date.rename(columns = {'year_x':'max','year_y':'min'}, inplace = True)
product_min_max_date.drop(['_merge'], axis=1, inplace =True)
product_min_max_date.reset_index(level=0, inplace=True)

# calculate diff between min and max
product_min_max_date = product_min_max_date.astype({'max': 'int', 'min': 'int'})
product_min_max_date['diff'] = (product_min_max_date['max'] - product_min_max_date['min'] + 1)

# final table
product_min_max_date

#### 4.3.4 Calculate Density

In [ ]:
def calculateDensity(x_axis, y_axis):
    x= x_axis
    y= y_axis

    # Calculate the point density
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    
    # Sort the points by density, so that the densest points are plotted last
    #idx = z.argsort()
    #x, y, z = x[idx], y[idx], z[idx]
    
    #return density
    return z

### 4.4 Merging For Luxury Beauty 

In [ ]:
#merging
#to find the intersection betweeen both tables
luxuryBeautyMergedInner = (pd.merge(luxuryBeautyProduct_table,luxuryBeautyReview_table, on='asin', how='inner', indicator=True)).set_index('asin')
luxuryBeautyMergedInner.reset_index(inplace = True)
luxuryBeautyMergedInner


#### 4.4.1 Final_luxuryBeautyReview_table

In [ ]:
final_luxuryBeautyReview_table = luxuryBeautyMergedInner.drop(['title','also_buy','rank','also_view','price','reviewerID', '_merge'],axis = 1)
final_luxuryBeautyReview_table

#### 4.4.2 merged_luxuryBeautyReview_table

In [ ]:
review_luxuryBeauty = final_luxuryBeautyReview_table[['asin', 'overall']]
luxuryBeautyProduct_rank = luxuryBeautyProduct_table[['asin','rank']]
merged_luxuryBeautyReview_table = process_merged_review(review_luxuryBeauty, luxuryBeautyProduct_rank)
merged_luxuryBeautyReview_table

#### 4.4.3 mv_luxuryBeautyReview_table

In [ ]:
luxuryBeautyVR = final_luxuryBeautyReview_table[['asin', 'overall', 'verified']]
mv_luxuryBeautyReview_table = process_verified_review(luxuryBeautyVR, merged_luxuryBeautyReview_table)
mv_luxuryBeautyReview_table

#### 4.4.4 Process for final luxury beauty table

In [ ]:
# suitable for passing merged_review_table or mv_review_table
def CalculateluxuryBeautyTimeWeightedAvg(rev_table):

    frt_copy = final_luxuryBeautyReview_table.copy()
    mmScaler = MinMaxScaler()
    frt_copy[['unixReviewTime']] = mmScaler.fit_transform(frt_copy[['unixReviewTime']])
    frt_copy['time-weighted-overall'] = frt_copy['overall'] * frt_copy['unixReviewTime'] 
    
    g = frt_copy[['asin', 'time-weighted-overall']]
    g = g.groupby(['asin'])['time-weighted-overall'].mean()
    time_score = pd.DataFrame(g)#, columns=['time-weighted-avg'])
    time_score.reset_index(inplace=True)
    
    rev_table = (pd.merge(rev_table, time_score, on='asin', how='inner', indicator=True)).set_index('asin')
    rev_table.reset_index(inplace = True)
    rev_table = rev_table.drop(['_merge'],axis = 1)

    return rev_table

luxuryBeautyReview_table_withTime = CalculateluxuryBeautyTimeWeightedAvg(mv_luxuryBeautyReview_table )
luxuryBeautyReview_table_withTime

### 4.5 Binning

#### 4.5.1 Binning for All Beauty Product

In [ ]:
bins = np.linspace(min(merged_review_table['rank']), max(merged_review_table['rank']), 5)
bins

In [ ]:
group_names = ['Very Good', 'Good', 'Moderate', 'Bad']
merged_review_table['rank-binned'] = pd.cut(merged_review_table['rank'], bins, labels=group_names, include_lowest=True)
merged_review_table.head()

In [ ]:
plt.figure(figsize = (5,4))
plt.bar(group_names, merged_review_table['rank-binned'].value_counts())
plt.title("Number of Product against Sales Rank")
plt.xlabel('Sales Rank')
plt.ylabel('Number of Product')

#### 4.5.2 Binning for Luxury Beauty Product

In [ ]:
bins_luxuryBeauty = np.linspace(min(luxuryBeautyReview_table_withTime['rank']), max(luxuryBeautyReview_table_withTime['rank']), 5)
rank_names = ['Very Good', 'Good', 'Moderate', 'Bad']
luxuryBeautyReview_table_withTime['rank-binned'] = pd.cut(luxuryBeautyReview_table_withTime['rank'], bins_luxuryBeauty, labels=rank_names, include_lowest=True)
luxuryBeautyReview_table_withTime

## 5.0 Data Analysis
****Important Note:****
All beauty productis our main dataset. This project is mainly analysing all beauty product dataset. Luxury beauty product is mainly used for predictive modelling section.

### 5.1 Corr: avg rating && sales rank

In [ ]:
plt.figure(figsize = (5,4))
bp = sns.boxplot(x='rank-binned', y='average-rating', data=merged_review_table)

In [ ]:
average_v_rank = sns.scatterplot(x='average-rating', 
                                 y='rank',
                                 data=merged_review_table)

average_v_rank.set(xlabel='Average Rating of Beauty Product', 
                   ylabel='Sales Rank', 
                   title="Average Rating of Beauty Products against Sales Rank")

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='average-rating', y='rank', data = merged_review_table, hue='rank-binned')

In [ ]:
num_review_am200 = merged_review_table.copy()
num_review_am200 = num_review_am200[num_review_am200['number-of-review']<= 200]


num_review_am200['average-rating-range'] = pd.cut(num_review_am200['average-rating'],
                                                  [0, 3, 4,6], 
                                                  right=False,
                                                  labels=["[1, 3)","[3, 4)","[4, 5]"])
num_review_am200

Unlike `merged_review_table`, the `num_review_am200` only contains product that has at most 200 reviews.



In [ ]:
#Finding correlation for 3 different categories of rating
low_rating = num_review_am200['average-rating-range'] == "[1, 3)"
mod_rating = num_review_am200['average-rating-range'] == "[3, 4)"
high_rating = num_review_am200['average-rating-range'] == "[4, 5]"

low_rating_table = num_review_am200[low_rating]
mod_rating_table = num_review_am200[mod_rating]
high_rating_table = num_review_am200[high_rating]
print(high_rating_table['average-rating'].corr(high_rating_table['rank']),"\n")
print(mod_rating_table['average-rating'].corr(mod_rating_table['rank']),"\n")
print(low_rating_table['average-rating'].corr(low_rating_table['rank']))

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='average-rating', y='rank', data=num_review_am200, hue='average-rating-range', palette=["k", "C3","C0"])

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='number-of-review', y='rank', data=num_review_am200, hue='average-rating-range', palette=["k", "C3","C0"])

In [ ]:
plt.subplot(3, 1, 1)
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='number-of-review', y='rank', data=low_rating_table, color="k")

plt.subplot(3, 1, 2)
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='number-of-review', y='rank', data=mod_rating_table, color="C3")

plt.subplot(3, 1, 3)
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='number-of-review', y='rank', data=high_rating_table, color="C0")

### 5.2 Corr: num review && sales rank

In [ ]:
num_review = merged_review_table[['asin','number-of-review', 'rank']]
num_review = num_review.sort_values(by=['rank'])
num_review

In [ ]:
num_review.corr()

In [ ]:
# sns.heatmap(num_review.corr(),cmap='jet')

In [ ]:
# figure, axes = plt.subplots(1, 2)
# num_review.plot(ax=axes[0])
# sns.scatterplot(x="number-of-review", y="rank", data=num_review)

In [ ]:
# plt.figure(figsize = (10,8))
# sns.regplot(x="number-of-review", y="rank", data=num_review)
# plt.ylim(0,)

#### 5.2.1 num review 4 bins

In [ ]:
# filter out num_review that are too high (consider them as outlier)
num_review_filter = num_review[num_review['number-of-review']<200]
num_review_filter.shape

# 32083(total all) 
# 32029(<500) / 54 (>=500)
# 31856(<200) / 227 (>=200)
# 31540(<100) / 543 (>=100)

In [ ]:
# square brackets '[]' mean the end point is included
# round parentheses '()' mean it's excluded
num_review_filter['num-review-range'] = pd.cut(num_review_filter['number-of-review'],
                                                  [0, 4, 7, 10, 200], 
                                                  right=True,
                                                  labels=['[1,4]', '(4,7]', '(7,10]', '(10,200)'])
num_review_filter

In [ ]:
num_review_filter['num-review-range'].value_counts().sort_index

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sttrplt=sns.scatterplot(x='number-of-review', y='rank', data = num_review_filter, hue='num-review-range')
plt.annotate(
# Label and coordinate
'Hard Candy Glow All The Way', xy=(110,0.86), xytext=(124, 0.9), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.15}
)
plt.annotate(
# Label and coordinate
'Death and Dentistry', xy=(122,0.82), xytext=(137, 0.86), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)
plt.annotate(
# Label and coordinate
'Norelco 8894 XL \nSpectra Shaver', xy=(107,0.64), xytext=(122, 0.70), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)
plt.annotate(
# Label and coordinate
'My New Pink \nButton - Audry', xy=(160,0.62), xytext=(175, 0.68), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)

sttrplt.set(xlabel='Total Number of Reviews', 
            ylabel='Sales Rank', 
            title="Total Number of Reviews against Sales Rank")


In [ ]:
num_review_filter[num_review_filter['number-of-review'] <=50].corr()

#### 5.2.2 num of high/low rating

In [ ]:
rating_grouping = final_review_table[['asin', 'overall']]

# count by number of high rating reviews
high_review_count = rating_grouping.groupby(['asin'])['overall'].apply(lambda x: (x >= 4).sum())
high_review_count = pd.DataFrame(high_review_count)
high_review_count.rename(columns={'overall':'number-of-high-rating-review'},inplace=True)
high_review_count.reset_index(inplace = True)

# count by number of low rating reviews
low_review_count = rating_grouping.groupby(['asin'])['overall'].apply(lambda x: (x < 4).sum())
low_review_count = pd.DataFrame(low_review_count)
low_review_count.rename(columns={'overall':'number-of-low-rating-review'},inplace=True)
low_review_count.reset_index(inplace = True)

In [ ]:
# m is a updated version of merged_review_table, with column recording the number of high & low rating reviews of each products
m = (pd.merge(high_review_count, low_review_count, on='asin', how='inner', indicator=True)).set_index('asin')
m.reset_index(inplace=True)
m = m.drop(['_merge'],axis = 1)
m

m = (pd.merge(merged_review_table, m, on='asin', how='inner', indicator=True)).set_index('asin')
m.reset_index(inplace=True)
m = m.drop(['_merge'],axis = 1)
m


In [ ]:
filter_num_review_50 = m['number-of-review'] <=50
m = m[filter_num_review_50]
m[['number-of-review','rank']].corr()

In [ ]:
nReview50 = sns.scatterplot(x="number-of-review", y="rank", data=m,color="blue")
nReview50.set(title="Number of Reviews(<=50) against Sales Rank")

In [ ]:
# ploting a graph to see the relationship between number of high rating reviews (those that are >= 4), and the sales rank
plt.subplot(2, 1, 1)
# getting those products that have at least one high rating review
m1 = m[m['number-of-high-rating-review'] > 0]
# we have 27858 products here
sns.scatterplot(x="number-of-high-rating-review", y="rank", data=m1)


# ploting a graph to see the relationship between number of low rating reviews (those that are < 4), and the sales rank
plt.subplot(2, 1, 2)
# getting those products that have at least one low rating review
m2 = m[m['number-of-low-rating-review'] > 0]
# we have 15950 products here
sns.scatterplot(x="number-of-low-rating-review", y="rank", data=m2)

#### 5.2.3 correlation at different year (2000-2018)

In [ ]:
# filter final_review_table, groupby asin and year
p_AvgRating_ByYear = final_review_table.groupby(['asin','year'], sort=True)['overall'].agg(['mean', 'count'])
p_AvgRating_ByYear = pd.DataFrame(p_AvgRating_ByYear)
p_AvgRating_ByYear.reset_index(level=['asin','year'], inplace=True)

# filter merged_review_table
p_rank = merged_review_table[['asin', 'rank']]

# merge both average rating and rank
avgRating_byYear = (pd.merge(p_rank, p_AvgRating_ByYear, on='asin', how='inner', indicator=True))
avgRating_byYear.drop(['_merge'], axis=1, inplace =True)
avgRating_byYear = avgRating_byYear.astype({'year': 'int'})
avgRating_byYear.sort_values(by=['year'], inplace=True)
avgRating_byYear = avgRating_byYear[['year','asin','rank','count','mean']]

avgRating_byYear

In [ ]:
# sort table according to rank, and reindex the rank
sorted_avgRating_byYear = avgRating_byYear.sort_values(by='rank')
sorted_avgRating_byYear.reset_index(drop=True,inplace=True)
sorted_avgRating_byYear['nrank'] = sorted_avgRating_byYear.index+1
sorted_avgRating_byYear

In [ ]:
frame_list =[]
density_list = []
date_list = []
for x in range(2000, 2019):
    tempPd = filterByYear(x, sorted_avgRating_byYear)
    density = calculateDensity(tempPd['count'], tempPd['nrank'])
    tempPd = pd.DataFrame(tempPd)
    frame_list.append(tempPd)
    density_list.append(density)
    date_list.append(str(x))

In [ ]:
#########
# If you're using Jupyter notebook
# Use this command to switch on matplotlib's backend, this will enable interactive UI

# %matplotlib notebook 

##########
# After running the animated gif
# use this command to switch off matplotlib's backend, this will disable interactive UI

# %matplotlib inline

#########
# To learn more about matplotlib
# %matplotlib --list

Animation of sales rank (y_axis) against number of reviews (x_axis) from 2000 to 2018 in every year 

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

ax.scatter([1,2000], [1,2000], c=[1,2000])
ax.set_title("title")
plt.xlabel("Number of Review")
plt.ylabel("Sales Rank")

#not in use (may omit)
def init():
    sc = ax.set_data([], [])
    return sc,

def animate(i):
    ax.scatter(frame_list[i]['count'], frame_list[i]['rank'], c=density_list[i], edgecolor='none')
    ax.set_title(date_list[i])
    
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=19, interval=1000)#, interval=100)

plt.show()     

### 5.3 Analysing "Very Good" Products

Further analysing product binned in "Very Good". These product ranking falls into the range 35 ~ 870386.75



In [ ]:
vg_filter = merged_review_table['rank'] <= 870386.75
vg_table = merged_review_table[vg_filter]
vg_table.sort_values(by=['rank'])

There are 11379 products that falls in the "Very Good" category, with the highest rank being 35, and the lowest rank is 870359.

We will further bin the products.

In [ ]:
bins_2 = np.linspace(min(vg_table['rank']), max(vg_table['rank']), 5)
bins_2

In [ ]:
group_names_2 = ['1', '2', '3', '4']
vg_table['upper-rank-binned'] = pd.cut(vg_table['rank'], bins_2, labels=group_names_2, include_lowest=True)
vg_table.head()

In [ ]:
plt.figure(figsize = (5,4))
plt.bar(group_names_2, vg_table['upper-rank-binned'].value_counts())

In [ ]:
plt.figure(figsize = (5,4))
bp = sns.boxplot(x='upper-rank-binned', y='average-rating', data=vg_table)

In [ ]:
vg_table.sort_values(by='average-rating')

In [ ]:
vg_table_lt200 = vg_table[vg_table['number-of-review']<200].copy() # cleared 214 products
vg_table_top = vg_table_lt200[vg_table['upper-rank-binned'] == '1'].copy()

vg_table_top['average-rating-range'] = pd.cut(vg_table_top['average-rating'],
                                                  [0, 4, 6], 
                                                  right=False,
                                                  labels=["[1, 4)", "[4, 5]"])
vg_table_top.head() 

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.scatterplot(x='average-rating', y='rank', data=vg_table_top, hue='average-rating-range', palette=["k", "C3"])

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.scatterplot(x='number-of-review', y='rank', data=vg_table_top, hue='average-rating-range', palette=["k", "C3"])

### 5.4 Slice Sales Rank < 5000



In [ ]:
#try to take data that have sales rank less than 50000
partial_table = merged_review_table[merged_review_table['rank'] <= 50000]

#min-max normalization
#partial_table['rank'] = (partial_table['rank'] - partial_table['rank'].min())/(partial_table['rank'].max()-partial_table['rank'].min())
partial_table

In [ ]:
partial_table.corr()

In [ ]:
num_review = partial_table[['asin','number-of-review', 'rank']]
num_review = num_review.sort_values(by=['rank'])
num_review

In [ ]:
plt.figure(figsize = (10,8))
sns.scatterplot(x="number-of-review", y="rank", data=num_review)

In [ ]:
plt.figure(figsize = (10,8))
average_v_rank = sns.scatterplot(x='average-rating', 
                                 y='rank',
                                 data=partial_table)

average_v_rank.set(xlabel='Average Rating of Beauty Product', 
                   ylabel='Sales Rank', 
                   title="Average Rating of Beauty Products against Sales Rank")

### 5.5 Price VS Sales Rank

In [ ]:
price_rank_table = product_table[['asin','rank','price']]
num_review_table = mv_review_table[['asin','number-of-review']]
# merge number of reviews
merged_price_table = (pd.merge(price_rank_table, num_review_table , on='asin', how='inner', indicator=True)).set_index('asin')
merged_price_table.reset_index(inplace = True)
merged_price_table = merged_price_table.drop(['_merge'],axis = 1)
merged_price_table.rename(columns={'overall':'number-of-review'},inplace=True)

# filter out product with 0 as sales rank 
#merged_price_table = merged_price_table[merged_price_table['rank'] > 0]
#merged_price_table = merged_price_table[merged_price_table['price'] > 0]

#remove outliers
price_z_scores = stats.zscore(pd.DataFrame(merged_price_table['price']))
price_abs_z_scores = np.abs(price_z_scores)
price_outlier = (price_abs_z_scores > 3).all(axis=1)
merged_price_table.drop(merged_price_table[price_outlier].index, inplace = True)
merged_price_table.sort_values(by=['rank']) 

In [ ]:
#min-max normalization
#price_rank_table['rank'] = (price_rank_table['rank'] - price_rank_table['rank'].min())/(price_rank_table['rank'].max()-price_rank_table['rank'].min())

In [ ]:
filter_num_review = merged_price_table['number-of-review'] < 50
#filter_price = merged_price_table['price'] < 25
merged_price_table = merged_price_table[filter_num_review]
#merged_price_table = merged_price_table[filter_price]
merged_price_table.sort_values(by=['price'])

In [ ]:
merged_price_table.corr()

In [ ]:
# square brackets '[]' mean the end point is included
# round parentheses '()' mean it's excluded
merged_price_table['price-range'] = pd.cut(merged_price_table['price'],
                                                  [-1, 25, 75, 125, 200], 
                                                  right=True,
                                                  labels=['[0,25]', '(25,75]', '(75,125]', '(125,200]'])
merged_price_table

In [ ]:
plt.figure(figsize = (15,10))
price_rank_graph = sns.scatterplot(x='price', 
                                 y='rank',
                                 data=merged_price_table, hue='price-range',palette=["k", "C3","C0","C2"])

price_rank_graph .set(xlabel='Price of Beauty Product', 
                   ylabel='Sales Rank', 
                   title="Price of Beauty Products against Sales Rank")

### 5.6 Analysing Review Upvotes

What 'vote' means? Check out
https://www.amazon.com/review/top-reviewer-faq.html

Summary: if people found your review helpful, they can give you a vote, if people found your review no helpful and decide to downvote you, your vote will get substracted

The vote score of each available product is calculated from all reviews given to it. From all of these reviews, only those with at least one vote are selected and applied weighted average to them.

Vote score formula: $$W=\frac {\sum_{i=1}^{n} w_i X_i} {\sum_{i=1}^{n} w_i}$$

$W$: weight average of the score (vote score)

$n$:	number of reviews to be averaged

$w_i$:	number of votes per review

$X_i$:	rating of the review



In [ ]:
# suitable for passing merged_review_table or mv_review_table
def calculateVoteScore(rev_table):
    
    g = final_review_table[['asin', 'overall', 'vote']].groupby(['asin']).apply(lambda x: (x['overall']*x['vote']).sum()/x['vote'].sum() if x['vote'].sum()!= 0 else 0)
    vote_score = pd.DataFrame(g, columns=['vote-score'])
    vote_score.reset_index(inplace=True)
    vote_score
    
    rev_table = (pd.merge(rev_table, vote_score, on='asin', how='inner', indicator=True)).set_index('asin')
    rev_table.reset_index(inplace = True)
    rev_table = rev_table.drop(['_merge'],axis = 1)
    # rev_table.rename(columns={'overall':'number-of-review'},inplace=True)

    return rev_table

In [ ]:
mv_review_table = calculateVoteScore(mv_review_table)


In [ ]:
new_mv_review_table = mv_review_table[mv_review_table['vote-score']>0]
new_mv_review_table

In [ ]:
corr_test = mv_review_table[['rank', 'average-rating', 'number-of-review', 'average-verified-rating', 'vote-score']].corr()
corr_test

In [ ]:
sns.heatmap(corr_test,square=True)

### 5.7 Analysing Time-Weighted-Average

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

# suitable for passing merged_review_table or mv_review_table
def CalculateTimeWeightedAvg(rev_table):

    frt_copy = final_review_table.copy()
    mmScaler = MinMaxScaler()
    frt_copy[['unixReviewTime']] = mmScaler.fit_transform(frt_copy[['unixReviewTime']])
    frt_copy['time-weighted-overall'] = frt_copy['overall'] * frt_copy['unixReviewTime'] 
    
    g = frt_copy[['asin', 'time-weighted-overall']]
    g = g.groupby(['asin'])['time-weighted-overall'].mean()
    time_score = pd.DataFrame(g)#, columns=['time-weighted-avg'])
    time_score.reset_index(inplace=True)
    
    rev_table = (pd.merge(rev_table, time_score, on='asin', how='inner', indicator=True)).set_index('asin')
    rev_table.reset_index(inplace = True)
    rev_table = rev_table.drop(['_merge'],axis = 1)

    return rev_table

In [ ]:
new_mv_review_table_withTime = CalculateTimeWeightedAvg(mv_review_table)
new_mv_review_table_withTime

### 5.8 Bivariate Attribute Analysis

#### 5.8.1 Sales Rank

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=mv_review_table["rank"])
bp.set(title='The Sales Rank of Beauty Products',
       xlabel='Sales Rank')

#### 5.8.2 Average Rating

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=mv_review_table["average-rating"])
bp.set(title='The Average Rating of Beauty Products',
       xlabel='Average Rating')

#### 5.8.3 Average Verified Rating

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=mv_review_table["average-verified-rating"], palette="Oranges_r")
bp.set(title='The Average Verified Rating of Beauty Products',
       xlabel='Average Verified Rating')

Comparing Average Rating and Average Verified Rating

In [ ]:
mv_review_table_combine=pd.melt(mv_review_table[['average-rating', 'average-verified-rating']])

In [ ]:
cbp=sns.boxplot(x="variable", y="value", data=mv_review_table_combine)
cbp.set(title='Comparison Between Average Rating and Average Verified Rating',
        xlabel='Types of Rating',
        ylabel='Value')

#### 5.8.4 Number of Reviews

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=mv_review_table["number-of-review"])
bp.set(title='The Number of Reviews of Beauty Products',
       xlabel='Number of Reviews')

#### 5.8.5 Vote Score 

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=new_mv_review_table["vote-score"], palette="Greens_r")
bp.set(title='The Vote Weighted Rating of Beauty Products',
       xlabel='Vote Weighted Rating')

#### 5.8.6 Time Weighted Overall 

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=new_mv_review_table_withTime["time-weighted-overall"], palette="Reds")
bp.set(title='The Time Weighted Rating of Beauty Products',
       xlabel='Time Weighted Rating')

In [ ]:
combined_rating=pd.melt(new_mv_review_table_withTime[['average-rating','average-verified-rating','vote-score','time-weighted-overall']])

In [ ]:
cbp=sns.boxplot(x="variable", y="value", data=combined_rating)
cbp.set(title='Comparison Between Average Rating and Time Weighted Rating',
        xlabel='Types of Rating',
        ylabel='Rating')

#### 5.8.7 Price 

In [ ]:
sns.set(rc={'figure.figsize':(10,5)})
bp = sns.boxplot(x=merged_price_table["price"])
bp.set(title='The Price of Beauty Products',
       xlabel='Price')

### 5.9 Correlation of All Attributes

In [ ]:
#preparing required table
partial_product_table = product_table[['asin','price']]

##Merging Price with new_mv_review_table_withTime
final_review_price_table = (pd.merge(new_mv_review_table_withTime, partial_product_table, on='asin', how='inner', indicator=True)).set_index('asin')
final_review_price_table.reset_index(inplace = True)
final_review_price_table = final_review_price_table .drop(['_merge'],axis = 1)
final_review_price_table

In [ ]:
final_review_price_table.describe()

In [ ]:
correlation = final_review_price_table.corr()

In [ ]:
sns.heatmap(correlation, square=True)

## 6.0 Data Mining

### 6.1 Association Rule Mining (ARM)

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### 6.1.1 Also Buy Products

In [ ]:
also_buy_table = product_table.drop(['title','also_view','rank','price'],axis = 1)
also_buy_table

In [ ]:
# separates each item in the list into a new row
also_buy_table = also_buy_table.explode('also_buy')
also_buy_table.reset_index(drop=True, inplace=True)

# cleaning up just to be safe
also_buy_table['also_buy'] = also_buy_table['also_buy'].str.strip()
also_buy_table.dropna(axis=0, subset=['also_buy'], inplace=True)
also_buy_table['also_buy'] = also_buy_table['also_buy'].astype('str')

# the product appear once in also buy (true, it was bought together)
also_buy_table['frequency'] = 1
also_buy_table

In [ ]:
# dropping products that does not exist in our product_table dataset
also_buy = also_buy_table[also_buy_table['also_buy'].isin(product_table['asin'])]
also_buy.reset_index(drop=True, inplace=True)
also_buy

In [ ]:
basket_ab = (also_buy.groupby(['asin', 'also_buy'])['frequency']
          .sum().unstack().reset_index().fillna(0)
          .set_index('asin')
         )
basket_ab

In [ ]:
# support 
frequent_itemsets = apriori(basket_ab, min_support=0.01, use_colnames=True)
frequent_itemsets

In [ ]:
# confidence rule
rules1_ab = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)
rules1_ab

In [ ]:
# lift rule
rules2_ab = association_rules(frequent_itemsets, metric="lift", min_threshold=0.01)
rules2_ab

These products are bought together

[B018J05XSQ]: Mia secret acrylic powder is highly adaptable for any nail tech experience level.

[B00C4207LY]: Nail Art Accessories Real Dry Dried Flowers 12 Colors Bundle Set in Wheel - Ready to Use by Winstonia

Both are nail products


#### 6.1.3 Also View Products

In [ ]:
also_view_table = product_table.drop(['title','also_buy','rank','price'],axis = 1)
also_view_table

In [ ]:
# Separates each item in the list into a new row
also_view_table = also_view_table.explode('also_view')
also_view_table.reset_index(drop=True, inplace=True)

# Cleaning up just to be safe
also_view_table['also_view'] = also_view_table['also_view'].str.strip()
also_view_table.dropna(axis=0, subset=['also_view'], inplace=True)
also_view_table['also_view'] = also_view_table['also_view'].astype('str')

# The product appear once in also buy (true, it was bought together)
also_view_table['frequency'] = 1
also_view_table = also_view_table[['asin', 'also_view','frequency']]
also_view_table

In [ ]:
# Dropping products that does not exist in our product_table dataset
also_view = also_view_table[also_view_table['also_view'].isin(product_table['asin'])]
also_view.reset_index(drop=True, inplace=True)
also_view

In [ ]:
basket_av = (also_view.groupby(['asin', 'also_view'])['frequency']
          .sum().unstack().reset_index().fillna(0)
          .set_index('asin')
         )
basket_av

In [ ]:
frequent_itemsets_2 = apriori(basket_av, min_support=0.01, use_colnames=True)
frequent_itemsets_2

In [ ]:
rules1_av = association_rules(frequent_itemsets_2, metric="confidence", min_threshold=0.001)
rules1_av

In [ ]:
rules2_av = association_rules(frequent_itemsets_2, metric="lift", min_threshold=0.001)
rules2_av

#### 6.1.3 Also Buy and Also View Frequency Analysis

Also Buy Products

In [ ]:
also_buy_f = also_buy.groupby(['also_buy']).count()
also_buy_f.rename(columns={'frequency':'also-buy-freq'},inplace=True)
also_buy_f.drop(['asin'], axis = 1, inplace=True)
also_buy_f.reset_index(inplace = True)
also_buy_f.rename(columns={'also_buy':'asin'},inplace=True)
also_buy_f

In [ ]:
mv2_review_table = (pd.merge(mv_review_table, also_buy_f, on='asin', how='left', indicator=True)).set_index('asin')
mv2_review_table.reset_index(inplace = True)
mv2_review_table['also-buy-freq'].fillna(0, inplace=True)
mv2_review_table.drop(['_merge'], axis=1, inplace=True)

mv2_review_table['also-buy-freq-range'] = pd.cut(mv2_review_table['also-buy-freq'],
                                                  [-1, 0, 10, 30], 
                                                  right=True,
                                                  labels=["0","(0, 10]","(10, 30]"])
mv2_review_table

In [ ]:
mv2_review_table["also-buy-freq-range"].value_counts()

Also View Products


In [ ]:
also_view_f = also_view.groupby(['also_view']).count()
also_view_f.rename(columns={'frequency':'also-view-freq'},inplace=True)
also_view_f.drop(['asin'], axis = 1, inplace=True)
also_view_f.reset_index(inplace = True)
also_view_f.rename(columns={'also_view':'asin'},inplace=True)
also_view_f

In [ ]:
mv3_review_table = (pd.merge(mv_review_table, also_view_f, on='asin', how='left', indicator=True)).set_index('asin')
mv3_review_table.reset_index(inplace = True)
mv3_review_table['also-view-freq'].fillna(0, inplace=True)
mv3_review_table.drop(['_merge'], axis=1, inplace=True)

mv3_review_table['also-view-freq-range'] = pd.cut(mv3_review_table['also-view-freq'],
                                                  [-1, 0, 10, 30], 
                                                  right=True,
                                                  labels=["0","(0, 10]","(10, 30]"])
mv3_review_table

In [ ]:
mv3_review_table["also-view-freq-range"].value_counts()

### 6.2 Predictive Modeling

#### 6.2.1 Predictive Model for All Beauty Product's Sales Rank

In [ ]:
#preparing required table
partial_product_table = product_table[['asin','price']]
#merging
merged_product_review_table = (pd.merge(merged_review_table, partial_product_table, on='asin', how='inner', indicator=True)).set_index('asin')
merged_product_review_table.reset_index(inplace = True)
merged_product_review_table = merged_product_review_table.drop(['_merge','rank'],axis = 1)
merged_product_review_table.rename(columns={'rank-binned':'rank'},inplace=True)

verified_rating_table = mv_review_table[['asin','average-verified-rating']][mv_review_table['average-verified-rating']>0]
merged_product_review_table = (pd.merge(merged_product_review_table, verified_rating_table, on='asin', how='inner', indicator=True)).set_index('asin')
merged_product_review_table.reset_index(inplace = True)
merged_product_review_table = merged_product_review_table.drop(['_merge','average-rating'],axis = 1)

time_score_table = new_mv_review_table_withTime[['asin','time-weighted-overall']]
merged_product_review_table = (pd.merge(merged_product_review_table, time_score_table, on='asin', how='inner', indicator=True)).set_index('asin')
merged_product_review_table.reset_index(inplace = True)
merged_product_review_table = merged_product_review_table.drop(['_merge','asin'],axis = 1)

#filtering
filter_price = merged_product_review_table['price'] > 0
#filter_num_review = merged_product_review_table['number-of-review'] <= 10
merged_product_review_table = merged_product_review_table[filter_price]
merged_product_review_table


In [ ]:
#normalization
product_review_table = merged_product_review_table.drop(['rank'],axis = 1)
product_review_table = (product_review_table - product_review_table.min())/(product_review_table.max()-product_review_table.min())
product_review_table['rank'] = merged_product_review_table['rank']

product_review_table

In [ ]:
product_review_table.corr()

In [ ]:
#splitting test size
X = product_review_table.drop("rank", axis=1)  
# copy the labels to another dataframe/series
y = product_review_table["rank"].copy()  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=1)


In [ ]:
#Model Training
model1 = MLPClassifier(hidden_layer_sizes =(4,5,3),max_iter=1000,random_state=1)
model1.fit(X_train,y_train)

model2 = KNeighborsClassifier(n_neighbors=20)
model2.fit(X_train,y_train)

In [ ]:
predictionMLP = model1.predict(X_test)
predictionKNC = model2.predict(X_test)


In [ ]:
print("The accuracy of model A is",accuracy_score(y_test, predictionMLP))
print("The accuracy of model B is",accuracy_score(y_test, predictionKNC))


In [ ]:
fig1,ax1 = plt.subplots(figsize = (20,10))
ax1.set_title("Confusion Matrix for Multilayer Perceptron Classifier(All Beauty Product)")
conf_matrix = confusion_matrix(y_test, predictionMLP)
mlp_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels = model1.classes_)
mlp_display.plot(cmap = 'Greens',xticks_rotation ='vertical',ax=ax1)

In [ ]:
fig2,ax2 = plt.subplots(figsize = (20,10))
ax2.set_title("Confusion Matrix for KNeighborsClassifier(All Beauty Product)")
conf_matrix = confusion_matrix(y_test, predictionKNC)
mlp_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels = model2.classes_)
mlp_display.plot(cmap = 'Greens',xticks_rotation ='vertical',ax=ax2)

In [ ]:
#ROC curve
encoder = LabelEncoder()
#predict probability
y_test = encoder.fit_transform(y_test)
pred_prob1 = model1.predict_proba(X_test)[:,1]
pred_prob2 = model2.predict_proba(X_test)[:,1]
# roc curve for models
fpr1, tpr1, thresh1 = roc_curve(y_test, pred_prob1, pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, pred_prob2, pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(y_test))]
p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)


In [ ]:
# matplotlib
plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='Multilayer Perceptron Classifier')
plt.plot(fpr2, tpr2, linestyle='--',color='green', label='KNN')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

It is evident from the plot that the AUC for the Multilayer Perceptron Classifier ROC curve is higher than that for the KNeighborsClassifier ROC curve. Multilayer Perceptron Classifier has a higher true positive rate than KNeighborsClassifier. Therefore, we can say that Multilayer Perceptron Classifier did a better job of classifying the positive class in the dataset. 

####6.2.2 Predictive Model for Luxury Beauty Product's Sales Rank

We tried to predict for luxury beauty product's sales rank in order to verify the effectiveness of review in predicting product's sales rank.

In [ ]:
#preparing required table
partial_luxuryBeautyProduct_table = luxuryBeautyProduct_table[['asin','price']]
#merging
merged_luxuryBeautyProduct_review_table = (pd.merge(luxuryBeautyReview_table_withTime, partial_luxuryBeautyProduct_table, on='asin', how='inner', indicator=True)).set_index('asin')
merged_luxuryBeautyProduct_review_table.reset_index(inplace = True)
merged_luxuryBeautyProduct_review_table  = merged_luxuryBeautyProduct_review_table .drop(['_merge','rank','asin','average-rating'],axis = 1)
merged_luxuryBeautyProduct_review_table.rename(columns={'rank-binned':'rank'},inplace=True)

#normalization
luxuryBeautyProduct_review_table = merged_luxuryBeautyProduct_review_table.drop(['rank'],axis = 1)
luxuryBeautyProduct_review_table= (luxuryBeautyProduct_review_table - luxuryBeautyProduct_review_table.min())/(luxuryBeautyProduct_review_table.max()-luxuryBeautyProduct_review_table.min())
luxuryBeautyProduct_review_table['rank'] = merged_luxuryBeautyProduct_review_table['rank']

luxuryBeautyProduct_review_table

In [ ]:
#splitting test size
XX = luxuryBeautyProduct_review_table.drop("rank", axis=1)  
# copy the labels to another dataframe/series
yy = luxuryBeautyProduct_review_table["rank"].copy()  
XX_train, XX_test, yy_train, yy_test = train_test_split(XX, yy, test_size=0.20,random_state=1)

#Model Training
model1.fit(XX_train,yy_train)

luxuryBeautyPredictionMLP = modelMLP.predict(XX_test)

print("The accuracy of model A is",accuracy_score(yy_test, luxuryBeautyPredictionMLP))

In [ ]:
fig,ax = plt.subplots(figsize = (20,10))
ax.set_title("Confusion Matrix for Multilayer Layer Perceptron Classifier(Luxury Beauty Product)")
mlp_conf_matrix = confusion_matrix(yy_test, luxuryBeautyPredictionMLP)
display = ConfusionMatrixDisplay(confusion_matrix=mlp_conf_matrix,display_labels = modelMLP.classes_)
display.plot(cmap = 'Greens',xticks_rotation ='vertical',ax=ax)

## 7.0 Data Visualization

### 7.1 Number of High Ratings VS Sales Rank
Will an increase in the number of high ratings (rating >= 4) affect the sales rank of a beauty product?


In [ ]:
mx = m.copy()
mx['ratio-high-to-total'] = mx['number-of-high-rating-review']/mx['number-of-review']
mx['average-rating-range'] = pd.cut(mx['average-rating'],
                                    [0, 3, 4, 6], 
                                    right=False,
                                    labels=["[1, 3)","[3, 4)","[4, 5]"])
mx

In [ ]:
mx.corr()

In [ ]:
plt.title('Ratio of High Rating Reviews against the Total Number of Reviews against the Sales Rank of a Beauty Product')
plt.ylabel('Sales Rank')
plt.xlabel('Ratio of High Rating Reviews against the Total Number of Reviews')
plot=sns.scatterplot(x="ratio-high-to-total", y="rank", data=mx, hue='average-rating-range', palette=["C0","C2","C3"])

The plotted graph is observed to have a weak negative association. This may be due to the imbalance of number of high rating reviews( 4) and number of low rating reviews(3). Although this graph showed a weak association between the two variables, we can observe that most of the points are distributed loosely at the lower right corner of the graph from the scatter plot above. This implies that the higher the ratio of the number of high rating reviews to the total number of reviews, the higher the sales rank the product is.

If a product receives a good amount of high rating reviews, the sales rank is believed to be better.


### 7.2 Average Rating VS Sales Rank 
What is the correlation between average rating of products and sales rank? Does higher average rating implies higher sales? 

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.scatterplot(x='average-rating', y='rank', data = merged_review_table, hue='rank-binned')

Based on our finding, the average rating of products has weak correlation with sales rank. This is because there are many confounding factors, as we need to take in consideration the time of the review, verification, content of the review as well. Simply take average rating cannot clearly imply the sales rank.


### 7.3 Number of Reviews VS Sales Rank
What is the correlation between number of reviews and sales rank? Does higher number of reviews implies higher sales rank?


In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sttrplt=sns.scatterplot(x='number-of-review', y='rank', data = num_review_filter, hue='num-review-range')
plt.annotate(
# Label and coordinate
'Hard Candy Glow All The Way', xy=(110,0.86), xytext=(124, 0.9), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.15}
)
plt.annotate(
# Label and coordinate
'Death and Dentistry', xy=(122,0.82), xytext=(137, 0.86), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)
plt.annotate(
# Label and coordinate
'Norelco 8894 XL \nSpectra Shaver', xy=(107,0.64), xytext=(122, 0.70), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)
plt.annotate(
# Label and coordinate
'My New Pink \nButton - Audry', xy=(160,0.62), xytext=(175, 0.68), color= "red", arrowprops={"facecolor": 'red', "shrink": 0.20}
)

sttrplt.set(xlabel='Total Number of Reviews', 
            ylabel='Sales Rank', 
            title="Total Number of Reviews against Sales Rank")


From the scatter plot above, the graph showed a moderate non linear association between the total number of reviews and sales rank. When the total number of reviews increases, the sales rank of the product will be better until the total number of reviews reaches approximately 50 reviews. After 50 number of reviews, the sales rank of the product started to remain constant. This can be concluded that 50 reviews are the optimal number of reviews for a beauty product in our case. However, there are some exceptions to some products which are located at the top right corner of the graph.


### 7.4 Price V.S. Sales Rank
What is the correlation between the pricing of products and sales rank? 

In [ ]:
plt.figure(figsize = (15,10))
price_rank_graph = sns.scatterplot(x='price', 
                                 y='rank',
                                 data=merged_price_table, hue='price-range',palette=["k", "C3","C0","C2"])

price_rank_graph .set(xlabel='Price of Beauty Product', 
                   ylabel='Sales Rank', 
                   title="Price of Beauty Products against Sales Rank")

Based on the scatter plot above, the price of products has a weak negative correlation with sales rank. This is because there are many confounding factors, as we need to take in consideration the time of the review, verification, content of the review as well. Simply taking the price of the product cannot clearly imply the sales rank. 



### 7.5 Review Up Votes V.S. Sales Rank
Will the number of review upvotes affect the number of buyers? 


In [ ]:
plt.figure(figsize = (15,10))
ax = sns.scatterplot(x='vote-score',y='rank',data=mv_review_table)

ax.set(xlabel='Vote Score', 
       ylabel='Sales Rank', 
       title="Vote Score of a Product against Sales Rank")

The vote score has -0.3511 of correlation value with sales rank. The higher the vote score, the lower (better) the sales rank. This shows that customers might somehow feel more “secure” to buy products that have a good up vote. The sense of having support from the community will enhance the chances of customers buying the product.

 ### 7.6 Timeframe and Reviews
 Do people in the current decade prioritise online reviews more compared to the last decade?

In [ ]:
timeframe = dfMergedInner[['year', 'month']].copy().groupby(['year']).count()
timeframe.rename(columns={'month':'number-of-review'},inplace=True)
timeframe.reset_index(inplace = True)
timeframe['cum-number-of-review'] = timeframe['number-of-review'].cumsum()
timeframe

In [ ]:
plt.figure(figsize = (10,8))
ax = sns.barplot(x="year", y="number-of-review", data=timeframe, color="C9", saturation=.5)

ax.set(title='Total Number of Reviews throughout the Years',
       xlabel='Year',
       ylabel='Total Number of Beauty Product Reviews')

In [ ]:
plt.figure(figsize = (10,8))
ax=sns.barplot(x="year", y="cum-number-of-review", data=timeframe, color="C9", saturation=.1)
sns.lineplot(x="year", y="cum-number-of-review", data=timeframe).plot(secondary_y=True)
ax.set(title='Cumulative Number of Reviews throughout the Years',
       xlabel='Year',
       ylabel='Cumulative Number of Beauty Product Reviews')

From the bar graph above, the bar chart shows a cumulative number of reviews for each year since 2000 to 2018. The total cumulative number of reviews increased steadily from the year 2000 to the year 2012. It began to go up dramatically ever since the year 2013, until the year 2017. In the end, the total cumulative number of reviews reached 357283 in the year 2018. We can observe that people in the last decade(year 2001 Jan 1 - 2009 Dec 31) make less reviews than the people in the current decade(year 2010 Jan 1 - 2019 Dec 31).

A conclusion can be drawn that people in the current decade prioritise online reviews more compared to the last decade.


### 7.7 Also Buy and Also View of the Product Reviews
Will a product with a high average rating appear more frequently in other products’ also_view / also_buy?


High Average Rating appearing more frequently in also_buy?

In [ ]:
sns.set(rc={'figure.figsize':(20,20)})
sns.scatterplot(x='average-rating', y='rank', data = mv2_review_table, hue='also-buy-freq-range', palette=["C9","C0","C3"])

This scatter plot shows the average rating of the beauty products against the sales rank. Each point is colored in a different hue to show the frequency of their appearance in other products’ also_buy section. It is shown that products with a high average rating do not appear very often in other products’ also_buy. However, those products that show up more in other products’ also_buy are scattered around different ranges of average rating.

From our dataset, it is clear that a product with high average rating does not appear more frequently in other products’ also_buy.


High Average Rating appearing more frequently in also_view?

In [ ]:
sns.set(rc={'figure.figsize':(20,20)})
sns.scatterplot(x='average-rating', y='rank', data = mv3_review_table, hue='also-view-freq-range', palette=["C9","C0","C3"])

This scatter plot shows the average rating of the beauty products against the sales rank. Each point is colored in a different hue to show the frequency of their appearance in other products’ also_view section. It is shown that products with a high average rating do not appear very often in other products’ also_view. However, those products that show up more in other products’ also_buy are scattered around different ranges of average rating.

From our dataset, it is clear that a product with a high average rating does not appear more frequently in other products’ also_view.


### 7.8 Prediction Accuracy for Luxury Beauty Products
Given the proposed hypothesis, the classification model is able to make correct predictions for the sales rank of beauty products with a high accuracy. Is this hypothesis also true for luxury beauty products? By using the same model, can we predict the sales rank of luxury beauty products with high accuracy as well?




In [ ]:
fig,ax = plt.subplots(figsize = (20,10))
ax.set_title("Confusion Matrix for Multilayer Layer Perceptron Classifier(Luxury Beauty Product)")
mlp_conf_matrix = confusion_matrix(yy_test, luxuryBeautyPredictionMLP)
display = ConfusionMatrixDisplay(confusion_matrix=mlp_conf_matrix,display_labels = modelMLP.classes_)
display.plot(cmap = 'Greens',xticks_rotation ='vertical',ax=ax)

The model, Multilayer Perceptron Classifier that we have built has been used to predict the sales rank of the luxury beauty product. In this study, we want to verify the effectiveness of this model  in predicting the sales rank of a product. From the previous study, an accuracy of 68% has been obtained in predicting the sales rank of all beauty products by using a Multilayer Perceptron Classifier with review data. We hypothesize that this model can be used  to predict sales rank of other products. Finally, the classification model has been tested using the prepared test sets and obtained an accuracy score of 75% for this model. Below showed the confusion matrix for the Multilayer Perceptron Classifier

Based on the confusion matrix, both of the models have a very high accuracy in predicting very good and good sales rank compared to other types of sales rank. There are no moderate and bad sales ranks that have been predicted correctly. This may be caused by the data imbalance issue in our dataset. Most of the luxury beauty products in our dataset have a very high sales rank, indicating that the products are selling well. This problem can be solved by using data augmentation to balance the dataset in the future. 

We can draw a conclusion that the hypothesis is correct. By using the same combination of descriptors which consists of review data, we are able to predict the sales rank of luxury beauty products with high accuracy of 75%.





## Conclusion

Our first finding is that sales rank is affected by numbers of reviews, high rating reviews, verified true reviews, voted review and timing of the reviews. This finding gives us a better view on how product reviews would actually affect sales. Marketing executives should spend efforts on getting quality reviews from customers as it will boost their product trustworthiness as well as increase sales indirectly. Beside that, marketing executives have to keep their review “updated” as well. Old reviews are less effective than latest reviews. 

Beside that we also built a model that could predict which product can potentially receive high sales rank based on its reviews. We developed a model as shown in 5.2 that has an accuracy of 68% by using a Multilayer Perceptron Classifier with review data after compared with KNeighborsClassifier which has lower value in our evaluation matrix, ROC. We used the same model and tuned again with its data on another category of beauty products called “Luxury Beauty” products and received the accuracy of 75%. This is a small leap of success for us to potentially predict which product has good potential according to their reviews.

Last but not least, we still have a huge room of improvement. Our future improvements should focus on continuously improving the accuracy of our model. Our data might be slightly skewed to high rating products. We might need to perform data augmentation to train for lower rating classification. Through the exploration of the data, we also identified a few interesting outliers that don't follow our models. Most of them are because they are not exactly the same category with the beauty product. Same some of them might have really outdated reviews.
